In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('POS_CASH_balance.csv')
pd.set_option('display.max_columns', None)

In [4]:
df.sample(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
5673198,2586990,281293,-36,12.0,7.0,Active,0.0,0.0
1148592,1339348,205588,-96,8.0,3.0,Active,0.0,0.0
1011523,2202839,352297,-11,24.0,13.0,Active,0.0,0.0
2909434,1894172,101190,-95,8.0,7.0,Active,0.0,0.0
1040621,1415091,405656,-16,3.0,0.0,Completed,0.0,0.0
2907798,1276574,325443,-44,18.0,13.0,Active,0.0,0.0
5583553,2831092,320919,-36,12.0,2.0,Active,0.0,0.0
3387097,2687520,216723,-2,12.0,9.0,Active,0.0,0.0
3899050,2094718,281690,-70,10.0,0.0,Active,0.0,0.0
3067307,1101049,254557,-50,11.0,3.0,Active,0.0,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6648649 entries, 0 to 6648648
Data columns (total 8 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   SK_ID_PREV             int64  
 1   SK_ID_CURR             int64  
 2   MONTHS_BALANCE         int64  
 3   CNT_INSTALMENT         float64
 4   CNT_INSTALMENT_FUTURE  float64
 5   NAME_CONTRACT_STATUS   object 
 6   SK_DPD                 float64
 7   SK_DPD_DEF             float64
dtypes: float64(4), int64(3), object(1)
memory usage: 405.8+ MB


In [6]:
pd.set_option('display.max_columns', None)

# Pilih kolom-kolom bertipe objek dan simpan dalam variabel 'categoric'
categoric = df.select_dtypes('object')

# Pilih kolom-kolom bertipe numerik dan simpan dalam variabel 'numeric'
numeric = df.select_dtypes('number')

# Tentukan jumlah baris dan kolom
rows = df.shape[0]
cols = df.shape[1]

# Cetak bentuk DataFrame
print(f'Jumlah Baris: {rows}\nJumlah Kolom: {cols}')

# Cetak ringkasan statistik data kategori
print('\nRingkasan Statistik Data Kategori')
display(categoric.describe().T)

# Cetak ringkasan statistik data numerik
print('\nRingkasan Statistik Data Numerik:')
display(numeric.describe().T)

Jumlah Baris: 6648649
Jumlah Kolom: 8

Ringkasan Statistik Data Kategori


,count,unique,top,freq
NAME_CONTRACT_STATUS,6648648,9,Active,6165636



Ringkasan Statistik Data Numerik:


,count,mean,std,min,25%,50%,75%,max
SK_ID_PREV,6648649.0,1.903777e+06,535672.381121,1000001.0,1434742.0,1897415.0,2369168.0,2843499.0
SK_ID_CURR,6648649.0,2.784379e+05,102748.290752,100001.0,189559.0,278668.0,367462.0,456255.0
MONTHS_BALANCE,6648649.0,-3.406381e+01,25.831071,-96.0,-50.0,-26.0,-14.0,-1.0
CNT_INSTALMENT,6635973.0,1.824699e+01,13.072527,1.0,10.0,12.0,24.0,92.0
CNT_INSTALMENT_FUTURE,6635984.0,1.185123e+01,12.058069,0.0,4.0,8.0,16.0,85.0
SK_DPD,6648648.0,1.268928e+00,39.455560,0.0,0.0,0.0,0.0,3959.0
SK_DPD_DEF,6648648.0,1.192462e-01,9.019553,0.0,0.0,0.0,0.0,2729.0


In [7]:
dfe = df.copy()
# Memeriksa baris duplikat
duplicate_rows = dfe.duplicated()

# Menampilkan jumlah baris duplikat
print(f"Jumlah baris duplikat: {duplicate_rows.sum()}")

# Fungsi untuk menghitung dan menampilkan kolom dengan nilai missing
def check_nan(dfe):
    # Hitung nilai null
    null_values = dfe.isnull().sum()

    # Buat DataFrame untuk total nilai null dan persentase
    nvc = pd.DataFrame(null_values[null_values > 0].sort_values(), columns=['Total Null Values'])  # Hanya ambil kolom dengan nilai null
    nvc['Percentage'] = (nvc['Total Null Values'] / dfe.shape[0]) * 100
    nvc["Data Type"] = [dfe[col].dtype for col in nvc.index]  # Ambil tipe data dari kolom yang ada

    # Urutkan berdasarkan total nilai null dan persentase
    nvc.sort_values(by=["Total Null Values", "Percentage"], ascending=False, inplace=True)

    # Tampilkan hasil dengan gaya
    return nvc.style.background_gradient(cmap='Blues')

# Panggil fungsi dan simpan hasilnya
styled_result = check_nan(dfe)
styled_result

Jumlah baris duplikat: 0


,Total Null Values,Percentage,Data Type
CNT_INSTALMENT,12676,0.190655,float64
CNT_INSTALMENT_FUTURE,12665,0.190490,float64
SK_DPD,1,0.000015,float64
NAME_CONTRACT_STATUS,1,0.000015,object
SK_DPD_DEF,1,0.000015,float64


jika kita tinjau bahwa2 kolom dengan data missing terbanyak yaitu CNT INSTALLMENT dan CNT_INSTALLMENT_FUTURE, kolom ini saling berkaitan yaitu mengenai termin kredit sebelumnya dengan sisa cicilan kredit sebelumnya


In [16]:
# Hitung rasio rata-rata CNT_INSTALMENT_FUTURE / CNT_INSTALMENT dari baris yang valid
valid_rows = dfe['CNT_INSTALMENT'].notnull() & dfe['CNT_INSTALMENT_FUTURE'].notnull()
ratio = (dfe.loc[valid_rows, 'CNT_INSTALMENT_FUTURE'] / dfe.loc[valid_rows, 'CNT_INSTALMENT']).mean()

# Imputasi CNT_INSTALMENT_FUTURE jika CNT_INSTALMENT tersedia
mask_future = dfe['CNT_INSTALMENT_FUTURE'].isnull() & dfe['CNT_INSTALMENT'].notnull()
dfe.loc[mask_future, 'CNT_INSTALMENT_FUTURE'] = dfe.loc[mask_future, 'CNT_INSTALMENT'] * ratio

# Imputasi CNT_INSTALMENT jika CNT_INSTALMENT_FUTURE tersedia
mask_instalment = dfe['CNT_INSTALMENT'].isnull() & dfe['CNT_INSTALMENT_FUTURE'].notnull()
dfe.loc[mask_instalment, 'CNT_INSTALMENT'] = dfe.loc[mask_instalment, 'CNT_INSTALMENT_FUTURE'] / ratio

# Sisa missing (jika masih ada) diisi dengan median
dfe['CNT_INSTALMENT'].fillna(dfe['CNT_INSTALMENT'].median(), inplace=True)
dfe['CNT_INSTALMENT_FUTURE'].fillna(dfe['CNT_INSTALMENT_FUTURE'].median(), inplace=True)

# Selanjutnya mengisi missing value dari sisa kolom yang ada
dfe['NAME_CONTRACT_STATUS'].fillna(dfe['NAME_CONTRACT_STATUS'].mode()[0], inplace=True)
dfe['SK_DPD'].fillna(dfe['SK_DPD'].median(), inplace=True)
dfe['SK_DPD_DEF'].fillna(dfe['SK_DPD_DEF'].median(), inplace=True)

In [17]:
dfe.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0.0,0.0
1,1715348,367990,-33,36.0,35.0,Active,0.0,0.0
2,1784872,397406,-32,12.0,9.0,Active,0.0,0.0
3,1903291,269225,-35,48.0,42.0,Active,0.0,0.0
4,2341044,334279,-35,36.0,35.0,Active,0.0,0.0


In [18]:
styled_result = check_nan(dfe)
styled_result

,Total Null Values,Percentage,Data Type


In [20]:
#simpan
dfe.to_parquet('pc_balance.parquet', index=False)